In [7]:
from smolgrad import Tensor
from smolgrad.core import no_grad
import smolgrad.nn as nn

In [8]:
model = nn.ModuleDict(dict(
    wte = nn.Linear(5, 10),
    h = nn.ModuleList([nn.Linear(10, 10) for _ in range(3)]),
    ln_f = nn.LayerNorm(10)
))
model.eval()

In [9]:
input_tensor = Tensor([[1.0, 2.0, 3.0, 4.0, 5.0], [2, 4, 5, 6, 5]])
print(input_tensor.shape)

with no_grad():
    output = model(input_tensor)

(2, 5)


In [10]:
output.shape

(2, 10)